### 아이디어 요약:
	•	전반전(First half) = “알고리즘이 왕” 모드
	→ 환경은 단순, 탭룰러 Q-learning 같은 전통 RL 알고리즘만으로 벤치마크를 오르는 게임
	•	후반전(Second half) = “평가/문제정의가 왕” 모드
	→ 추론(Reasoning)을 하나의 ‘행동(Action)’으로 넣어서, 사전지식(priors)·계획(planning)·테스트시점 계산(test-time compute)을 활용

아래 예제는 그리드 미로 환경에서 </br>
	1. 전반전: 순수 Q-러닝으로 목표 도달 </br>
	2. 후반전: THINK(추론) → ACT(행동) 루프를 도입해 “열쇠→문→목표” 순서를 스스로 계획해서 해결 </br>
을 비교합니다.

1) 공용 환경: 아주 작은 GridWorld
	-	.: 빈칸, #: 벽, K: 열쇠, D: 문(열쇠 없으면 못 지나감), G: 목표, S: 시작
	-	행동: 위/아래/왼/오 + (후반전에서만) THINK
	-	보상: 한 스텝당 −0.01, 열쇠 획득 시 +0.1, 목표 도달 시 +1.0, 스텝 제한 초과 시 종료

In [1]:
# gridworld_rl.py

import random
from collections import defaultdict, deque
from typing import Tuple, List, Optional

UP, DOWN, LEFT, RIGHT, THINK = range(5)
ACTIONS = [UP, DOWN, LEFT, RIGHT]
ACTION_NAMES = {UP:"UP", DOWN:"DOWN", LEFT:"LEFT", RIGHT:"RIGHT", THINK:"THINK"}

class GridWorld:
    def __init__(self, grid: List[str], start: Tuple[int,int], goal: Tuple[int,int],
                 key: Optional[Tuple[int,int]]=None, door: Optional[Tuple[int,int]]=None, max_steps: int=200):
        self.grid = [list(row) for row in grid]
        self.h = len(grid)
        self.w = len(grid[0])
        self.start = start
        self.goal = goal
        self.key = key
        self.door = door
        self.has_key = False
        self.max_steps = max_steps
        self.reset()
        
    def reset(self):
        self.x, self.y = self.start
        self.has_key = False
        self.steps = 0
        return self._get_state()
    
    def _get_state(self):
        return (self.x, self.y, int(self.has_key))
    
    def in_bounds(self, x,y):
        return 0<=x<self.w and 0<=y<self.h
    
    def is_wall(self, x,y):
        return self.grid[y][x] == '#'
    
    def step(self, action: int):
        reward = -0.01
        done = False
        info = {}
        self.steps += 1
        if action == THINK:
            # THINK는 외부 세계를 바꾸지 않음(후반전에서 내부 계획에 쓰입니다)
            return self._get_state(), -0.005, False, info
        
        dx, dy = 0,0
        if action == UP: dy = -1
        elif action == DOWN: dy = 1
        elif action == LEFT: dx = -1
        elif action == RIGHT: dx = 1
        
        nx, ny = self.x + dx, self.y + dy
        if self.in_bounds(nx, ny) and not self.is_wall(nx, ny):
            # 문은 열쇠 없으면 통과 불가
            if self.door is not None and (nx, ny) == self.door and not self.has_key:
                reward -= 0.02  # 문에 막힘
            else:
                self.x, self.y = nx, ny
        
        # 열쇠 획득
        if self.key is not None and (self.x, self.y) == self.key and not self.has_key:
            self.has_key = True
            reward += 0.1
        
        # 목표 도달(문이 있을 경우 열쇠 소지 필요)
        if (self.x, self.y) == self.goal and (self.door is None or self.has_key):
            reward += 1.0
            done = True
        
        if self.steps >= self.max_steps:
            done = True
        return self._get_state(), reward, done, info
    
    def render(self):
        out = []
        for y in range(self.h):
            row=""
            for x in range(self.w):
                if (x,y) == (self.x,self.y):
                    row+="A"
                elif (x,y) == self.goal:
                    row+="G"
                elif self.key is not None and (x,y) == self.key and not self.has_key:
                    row+="K"
                elif self.door is not None and (x,y) == self.door:
                    row+="D"
                else:
                    row+=self.grid[y][x]
            out.append(row)
        return "\n".join(out)

def make_simple_maze():
    grid = [
        ".....",
        ".###.",
        ".#..#",
        ".#.#.",
        ".....",
    ]
    start=(0,0); goal=(4,4)
    return GridWorld(grid, start, goal, None, None, max_steps=200)

def make_door_maze():
    # 열쇠를 반드시 먼저 주워야 문을 넘어 목표로 갈 수 있는 구조(조금 더 어렵게)
    grid = [
        "#########",
        "#S....#G#",
        "#.##.#..#",
        "#.#D.#..#",
        "#.#..#..#",
        "#.####..#",
        "#..K....#",
        "#########",
    ]
    start=goal=key=door=None
    gridL = list(grid)
    for y,row in enumerate(gridL):
        for x,ch in enumerate(row):
            if ch=='S': start=(x,y); gridL[y]=gridL[y].replace('S','.')
            if ch=='G': goal=(x,y); gridL[y]=gridL[y].replace('G','.')
            if ch=='K': key=(x,y);  gridL[y]=gridL[y].replace('K','.')
            if ch=='D': door=(x,y); gridL[y]=gridL[y].replace('D','.')
    return GridWorld(gridL, start, goal, key, door, max_steps=400)

2) 전반전: “알고리즘 중심” — 탭룰러 Q-러닝

핵심 포인트
- 상태(state) = (x, y, has_key) 
- 행동(action) = UP, DOWN, LEFT, RIGHT 
- 정책: ε-탐욕(epsilon-greedy)
- 업데이트: Q[s,a] ← Q[s,a] + α*(r + γ*max_a' Q[s',a'] − Q[s,a])

In [2]:
# 이어서 같은 파일에 붙여넣기

class QLearningAgent:
    def __init__(self, actions, alpha=0.2, gamma=0.99, epsilon=0.2):
        self.Q = defaultdict(float)
        self.actions = actions
        self.alpha=alpha
        self.gamma=gamma
        self.epsilon=epsilon
    
    def _key(self, state, action): return (state, action)
    
    def choose_action(self, state):
        if random.random() < self.epsilon:
            return random.choice(self.actions)
        vals = [(self.Q[self._key(state,a)], a) for a in self.actions]
        return max(vals)[1]
    
    def update(self, state, action, reward, next_state):
        key = self._key(state, action)
        max_next = max(self.Q[(next_state,a)] for a in self.actions)
        self.Q[key] += self.alpha * (reward + self.gamma*max_next - self.Q[key])

def train_qlearning(env_fn, episodes=200, seed=0):
    random.seed(seed)
    env=env_fn()
    agent=QLearningAgent(actions=ACTIONS)
    successes=0; total_steps=0
    for ep in range(episodes):
        s=env.reset()
        done=False; steps=0
        while not done:
            a = agent.choose_action(s)
            ns, r, done, _ = env.step(a)
            agent.update(s,a,r,ns)
            s = ns; steps+=1
            if steps>env.max_steps: break
        total_steps += steps
        if (env.x,env.y)==env.goal and (env.door is None or env.has_key):
            successes+=1
        if (ep+1)%50==0:
            print(f"[Q] ep {ep+1}: success {successes/(ep+1):.2f}, avg steps {total_steps/(ep+1):.1f}")
    return agent

if __name__ == "__main__":
    print("== Simple maze (전반전: 순수 Q-learning) ==")
    _ = train_qlearning(make_simple_maze, episodes=200, seed=42)

    print("\n== Door maze (전반전: 순수 Q-learning) ==")
    _ = train_qlearning(make_door_maze, episodes=200, seed=42)

== Simple maze (전반전: 순수 Q-learning) ==
[Q] ep 50: success 0.98, avg steps 23.6
[Q] ep 100: success 0.99, avg steps 17.1
[Q] ep 150: success 0.99, avg steps 14.7
[Q] ep 200: success 0.99, avg steps 13.9

== Door maze (전반전: 순수 Q-learning) ==
[Q] ep 50: success 0.96, avg steps 74.7
[Q] ep 100: success 0.98, avg steps 47.7
[Q] ep 150: success 0.99, avg steps 38.4
[Q] ep 200: success 0.99, avg steps 33.9


3) 후반전: “추론을 행동으로” — ReAct 스타일 미니 에이전트

핵심 포인트
	•	THINK 라는 내부 행동을 추가: 외부 세계는 안 바꾸고, 계획(plan) 을 업데이트
	•	사전지식(priors)의 역할을 계획/추론으로 형상화
	•	여기서는 가르치는 목적상 BFS 계획자를 사용(“열쇠→문→목표” 순서로 경로 계획)
	•	현실의 LLM-기반 에이전트에서는 언어 priors + 추론이 이 자리를 차지

In [3]:
# 같은 파일 계속

class ReActAgent:
    """
    아주 단순화한 'Reason + Act' 에이전트
    - THINK: 현재 상태에서 '열쇠(필요시) -> 목표' 로 가는 계획(Path)을 BFS로 계산
    - ACT: 계획에 따라 한 스텝씩 이동
    """
    def __init__(self, use_memory: bool=False):
        self.plan: List[int] = []
        self.use_memory = use_memory
        self.memory = {}  # (맵 서명) -> plan
    
    def _env_id(self, env: GridWorld):
        return tuple(tuple(row) for row in env.grid), env.start, env.goal, env.key, env.door
    
    def _neighbors(self, env: GridWorld, x,y, has_key):
        for a,(dx,dy) in {UP:(0,-1), DOWN:(0,1), LEFT:(-1,0), RIGHT:(1,0)}.items():
            nx, ny = x+dx, y+dy
            if not env.in_bounds(nx,ny) or env.is_wall(nx,ny):
                continue
            if env.door and (nx,ny)==env.door and not has_key:
                continue
            yield a, nx, ny
    
    def _bfs(self, env: GridWorld, start: Tuple[int,int], goal: Tuple[int,int], has_key: bool):
        q=deque([ (start, []) ])
        seen=set([start])
        while q:
            (x,y), path = q.popleft()
            if (x,y)==goal:
                return path
            for a,nx,ny in self._neighbors(env,x,y,has_key):
                if (nx,ny) in seen: continue
                seen.add((nx,ny))
                q.append(((nx,ny), path+[a]))
        return None
    
    def think(self, env: GridWorld):
        # (가르침용 단순화) 지금 위치에서 '열쇠(필요시) -> 목표'로 가는 전체 경로를 만든다
        plan=[]
        x,y = env.x, env.y
        has_key = env.has_key
        if env.key and not has_key:
            p1 = self._bfs(env, (x,y), env.key, has_key)
            if p1 is None: self.plan=[]; return
            plan += p1
            has_key = True
            x,y = env.key
        p2 = self._bfs(env, (x,y), env.goal, has_key)
        if p2 is None: self.plan=[]; return
        plan += p2
        self.plan = plan
        if self.use_memory:
            self.memory[self._env_id(env)] = list(plan)
    
    def act(self, env: GridWorld):
        if not self.plan:
            return THINK
        a = self.plan.pop(0)
        return a

def run_react(env_fn, episodes=3, seed=0, use_memory=False):
    random.seed(seed)
    env=env_fn()
    agent=ReActAgent(use_memory=use_memory)
    for ep in range(episodes):
        env.reset()
        steps=0; done=False; total_reward=0.0
        agent.think(env)  # 처음에 한 번 생각
        while not done and steps<env.max_steps:
            a = agent.act(env)  # 계획이 없으면 THINK가 나오고, 있으면 한 칸 이동
            ns,r,done,_ = env.step(a)
            total_reward += r
            # 열쇠를 집거나 계획이 소진되면 다시 THINK로 재계획
            if a==THINK or not agent.plan or (env.key and (env.x,env.y)==env.key):
                agent.think(env)
            steps+=1
        print(f"[ReAct] ep {ep+1}: steps={steps}, reward={total_reward:.2f}, success={done and (env.x,env.y)==env.goal}")

if __name__ == "__main__":
    print("\n== Door maze (후반전: ReAct - THINK + ACT) ==")
    run_react(make_door_maze, episodes=3, seed=0)


== Door maze (후반전: ReAct - THINK + ACT) ==
[ReAct] ep 1: steps=16, reward=0.94, success=True
[ReAct] ep 2: steps=16, reward=0.94, success=True
[ReAct] ep 3: steps=16, reward=0.94, success=True


실행하면 보통:
	•	ReAct 에이전트는 첫 에피소드부터 바로 성공하고, 스텝 수도 짧습니다(예: ~16스텝).
	•	THINK는 외부 세계를 안 바꾸는 내부 행동이고, 그 결과로 계획(plan) 이 갱신됩니다.
→ 이게 글에서 말하는 “추론을 행동으로(reasoning as an action)”의 핵심 아이디어예요.

후반전 관점: “이미 잘 되는 레시피(프리트레이닝+스케일+추론)를 전제로, 과제를 어떻게 정의/평가하느냐가 핵심이며, 추론/메모리/상호작용을 평가 안에 넣어 실제 유용성(utility)을 끌어올린다.”